In [1]:
import cv2

# Open two video capture objects (assuming two cameras are available)
cap1 = cv2.VideoCapture(0)
cap2 = cv2.VideoCapture(1)

if not cap1.isOpened() or not cap2.isOpened():
    print("Error: Could not open one or both cameras.")
    cap1.release()
    cap2.release()
    exit()

while True:
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()

    if not ret1 or not ret2:
        print("Error: Failed to read from one or both cameras.")
        break

    cv2.imshow('Camera 1', frame1)
    cv2.imshow('Camera 2', frame2)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap1.release()
cap2.release()
cv2.destroyAllWindows()


In [4]:
from ultralytics import YOLO
import cv2

# Load the YOLOv8 model for pose estimation
model = YOLO("yolo11n-pose.pt")

# Open two webcam streams
cap1 = cv2.VideoCapture(0)
cap2 = cv2.VideoCapture(1)

# Check if both cameras opened successfully
if not cap1.isOpened() or not cap2.isOpened():
    print("Error: Cannot open one or both webcams.")
    cap1.release()
    cap2.release()
    exit()

while True:
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()

    if not ret1 or not ret2:
        print("Error: Cannot read from one or both cameras.")
        break

    # Run pose estimation on both frames
    results1 = model(frame1)
    results2 = model(frame2)

    # Plot the results
    annotated_frame1 = results1[0].plot()
    annotated_frame2 = results2[0].plot()

    # Display both frames
    cv2.imshow("Camera 1 - Pose Estimation", annotated_frame1)
    cv2.imshow("Camera 2 - Pose Estimation", annotated_frame2)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cap1.release()
cap2.release()
cv2.destroyAllWindows()



0: 480x640 1 person, 13.0ms
Speed: 3.3ms preprocess, 13.0ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 8.9ms
Speed: 1.3ms preprocess, 8.9ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 9.6ms
Speed: 1.9ms preprocess, 9.6ms inference, 2.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 8.5ms
Speed: 1.8ms preprocess, 8.5ms inference, 4.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 10.7ms
Speed: 1.3ms preprocess, 10.7ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 11.9ms
Speed: 1.5ms preprocess, 11.9ms inference, 2.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 9.4ms
Speed: 1.5ms preprocess, 9.4ms inference, 3.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 persons, 12.8ms
Speed: 3.7ms preprocess, 12.8ms inference, 2.7ms postprocess per image at shape (1, 3, 480, 6

In [3]:
import cv2

# Open two video capture objects
cap1 = cv2.VideoCapture(0)  # First camera
cap2 = cv2.VideoCapture(1)  # Second camera

# Check if cameras opened successfully
if not cap1.isOpened():
    print("Error: Could not open camera 0.")
    exit()
if not cap2.isOpened():
    print("Error: Could not open camera 1.")
    exit()

while True:
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()

    if not ret1 or not ret2:
        print("Error: Could not read frame from one or both cameras.")
        break

    # Optionally resize frames to same size for display
    frame1 = cv2.resize(frame1, (640, 480))
    frame2 = cv2.resize(frame2, (640, 480))

    # Combine the two frames side by side
    combined = cv2.hconcat([frame1, frame2])

    # Show combined frame
    cv2.imshow('Camera 0 (left) | Camera 1 (right)', combined)

    # Exit loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release everything
cap1.release()
cap2.release()
cv2.destroyAllWindows()


Rotation (R):
 [[ 0.98694128  0.05045164  0.15297563]
 [-0.001877    0.95322218 -0.30226469]
 [-0.16106951  0.29803037  0.94086902]]
Translation (t):
 [[-0.37913577]
 [ 0.92466243]
 [ 0.03543246]]


In [4]:
import cv2
import numpy as np

# Replace this with your actual camera intrinsic matrix (calibrated values)
# [ fx   0  cx ]
# [  0  fy  cy ]
# [  0   0   1 ]
K = np.array([[700, 0, 320],
              [0, 700, 240],
              [0,   0,   1]])

# Capture from both cameras
cap1 = cv2.VideoCapture(0)
cap2 = cv2.VideoCapture(1)

if not cap1.isOpened() or not cap2.isOpened():
    print("Error: Cannot open one or both cameras.")
    exit()

# Capture one frame from each
ret1, frame1 = cap1.read()
ret2, frame2 = cap2.read()
cap1.release()
cap2.release()

if not ret1 or not ret2:
    print("Error: Cannot read frames.")
    exit()

# Convert to grayscale
gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)

# SIFT detector
sift = cv2.SIFT_create()

# Detect and compute keypoints/descriptors
kp1, des1 = sift.detectAndCompute(gray1, None)
kp2, des2 = sift.detectAndCompute(gray2, None)

# Match features using FLANN
index_params = dict(algorithm=1, trees=5)  # KDTree
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1, des2, k=2)

# Ratio test as per Lowe's paper
good_matches = []
pts1, pts2 = [], []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good_matches.append(m)
        pts1.append(kp1[m.queryIdx].pt)
        pts2.append(kp2[m.trainIdx].pt)

pts1 = np.float32(pts1)
pts2 = np.float32(pts2)

if len(pts1) < 8:
    print("Not enough matches found.")
    exit()

# Estimate fundamental matrix with RANSAC
F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC)

# Essential matrix
E = K.T @ F @ K

# Recover relative pose
_, R, t, _ = cv2.recoverPose(E, pts1, pts2, K)

print("Rotation (R):\n", R)
print("Translation (t):\n", t)

# Draw matches
match_img = cv2.drawMatches(frame1, kp1, frame2, kp2, good_matches, None,
                            flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
cv2.imshow("Matches", match_img)
cv2.waitKey(0)
cv2.destroyAllWindows()


Rotation (R):
 [[ 0.99780481 -0.05315402  0.03949962]
 [ 0.05857395  0.98665631 -0.15191583]
 [-0.03089761  0.15389599  0.98760385]]
Translation (t):
 [[-0.16297526]
 [ 0.13326154]
 [-0.97758909]]


In [5]:
dist1 = np.zeros(5)
dist2 = np.zeros(5)